In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.select import Select
import requests
import os
from PIL import Image

In [2]:
url = "https://readcomiconline.li/Comic/Injustice-Gods-Among-Us-I"
parsed_url = url.split("/")
comic_series = parsed_url[-1]

In [3]:


options = webdriver.FirefoxOptions()
options = options.add_argument("-headless")
driver = webdriver.Firefox(options=options)
driver.get(url)
html_content = driver.page_source



In [4]:
table_element = driver.find_element(By.CLASS_NAME, "listing")
anchor_elements = table_element.find_elements(By.TAG_NAME, "a")
# issues_urls = [elem.get_attribute("href") for elem in anchor_elements]
issues = {elem.text: elem.get_attribute("href") for elem in anchor_elements}




driver.quit()


In [5]:
comic_series_folder = f"{comic_series}"
try:
    os.makedirs(comic_series_folder)
    print(f"Directory '{comic_series_folder}' created successfully.")
    comic_series_path = os.path.join(comic_series_folder)
except OSError as e:
    print(f"Error creating directory '{comic_series_folder}': {e}")
    # Optionally, handle the error or raise it to the caller.


Directory 'Injustice-Gods-Among-Us-I' created successfully.


In [6]:
for title, url in issues.items():
    print(title, url)

Injustice: Gods Among Us [I] Issue #36 https://readcomiconline.li/Comic/Injustice-Gods-Among-Us-I/Issue-36?id=25113
Injustice: Gods Among Us [I] Issue #35 https://readcomiconline.li/Comic/Injustice-Gods-Among-Us-I/Issue-35?id=25111
Injustice: Gods Among Us [I] Issue #34 https://readcomiconline.li/Comic/Injustice-Gods-Among-Us-I/Issue-34?id=25109
Injustice: Gods Among Us [I] Issue #33 https://readcomiconline.li/Comic/Injustice-Gods-Among-Us-I/Issue-33?id=25107
Injustice: Gods Among Us [I] Issue #32 https://readcomiconline.li/Comic/Injustice-Gods-Among-Us-I/Issue-32?id=25106
Injustice: Gods Among Us [I] Issue #31 https://readcomiconline.li/Comic/Injustice-Gods-Among-Us-I/Issue-31?id=25104
Injustice: Gods Among Us [I] Issue #30 https://readcomiconline.li/Comic/Injustice-Gods-Among-Us-I/Issue-30?id=25102
Injustice: Gods Among Us [I] Issue #29 https://readcomiconline.li/Comic/Injustice-Gods-Among-Us-I/Issue-29?id=25099
Injustice: Gods Among Us [I] Issue #28 https://readcomiconline.li/Comic/

Injustice Gods Among Us [I] Issue #36


In [37]:
for title, url in issues.items():
    # if title[-2] != "#" and int(title[-2:]) > 7:
    if int(title[-1]) <= 2 and title[-2] == "#":
    # if int(title[-2:]) > 22:
        # pass
    # else:
        try:
            pattern = r'[<>:"\/|?*]'
            # Use re.sub() to replace all occurrences of the special characters with an empty string
            title = re.sub(pattern, '', title)

            issue_folder = os.path.join(comic_series_path, title)
            images_folder = os.path.join(issue_folder, "images")
            os.makedirs(images_folder)
            print(f"Directories '{issue_folder}' and '{images_folder}' created successfully.")
        except OSError as e:
            print(f"Error creating directories: {e}")
            # Optionally, handle the error or raise it to the caller.

        print("Trying", title)

        options = webdriver.FirefoxOptions()
        options = options.add_argument("-headless")
        driver = webdriver.Firefox(options=options)
        driver.get(url)
        select_element_page = driver.find_element(By.ID, "selectReadType")
        select = Select(select_element_page)
        select.select_by_value("1")

        div_image = driver.find_element(By.ID, "divImage")
        img_elements = div_image.find_elements(By.TAG_NAME, "img")

        img_srcs = [elem.get_attribute("src") for elem in img_elements]

        driver.quit()

        image_files = []
        i = 1
        for img_url in img_srcs:
            response = requests.get(img_url)
            if response.status_code == 200:
                image_name = f"{title}_image_{i}.jpg"
                with open(os.path.join(images_folder, image_name), "wb") as image:
                    image_download = response.content
                    image.write(image_download)
                pil_image = Image.open(os.path.join(images_folder, image_name))
                image_files.append(pil_image)
            i += 1
        image_files[0].save(f"{comic_series_folder}/{title}.pdf", save_all=True, append_images=image_files[1:])
        print("Successful", title)


Error creating directories: [WinError 183] Cannot create a file when that file already exists: 'Injustice-Gods-Among-Us-I\\Injustice Gods Among Us [I] Issue #2\\images'
Trying Injustice Gods Among Us [I] Issue #2
Successful Injustice Gods Among Us [I] Issue #2
Directories 'Injustice-Gods-Among-Us-I\Injustice Gods Among Us [I] Issue #1' and 'Injustice-Gods-Among-Us-I\Injustice Gods Among Us [I] Issue #1\images' created successfully.
Trying Injustice Gods Among Us [I] Issue #1
Successful Injustice Gods Among Us [I] Issue #1
